In [0]:
import requests
import os
import zipfile
import io

zip_url = "https://github.com/chadwickbureau/register/archive/refs/heads/master.zip"
landing_path = "/Volumes/mlb/00_landing/data_sources/player_name_lookup"
os.makedirs(landing_path, exist_ok=True)

try:
    response = requests.get(zip_url, timeout=30)
    response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        
        for file_info in csv_files:
            filename = os.path.basename(file_info)
            if not filename: continue 
            
            output_full_path = os.path.join(landing_path, filename)
            with z.open(file_info) as source, open(output_full_path, "wb") as target:
                target.write(source.read())
    
    dbutils.jobs.taskValues.set(key="continue_downstream", value="yes")

except Exception as e:
    dbutils.jobs.taskValues.set(key="continue_downstream", value="no")